In [1]:
!pip install spacy

In [2]:
%reload_ext nb_black
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
import spacy
import re

import warnings

warnings.filterwarnings(action="ignore")



<IPython.core.display.Javascript object>

In [3]:
!python -m spacy download en

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
C:\Users\jlim7\anaconda3\lib\site-packages\en_core_web_sm -->
C:\Users\jlim7\anaconda3\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You do not have sufficient privilege to perform this operation.


<IPython.core.display.Javascript object>

In [4]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "cornell_movie_dialogs"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

dialogs_df = pd.read_sql_query("select * from dialogs", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


dialogs_df.head(10)

,index,dialogs
0,0,Can we make this quick? Roxanne Korrine and A...
1,1,"Well, I thought we'd start with pronunciation,..."
2,2,Not the hacking and gagging and spitting part....
3,3,Okay... then how 'bout we try out some French ...
4,4,You're asking me out. That's so cute. What's ...
5,5,Forget it.
6,6,"No, no, it's my fault -- we didn't have a prop..."
7,7,Cameron.
8,8,"The thing is, Cameron -- I'm at the mercy of a..."
9,9,Seems like she could get a date easy enough...


<IPython.core.display.Javascript object>

In [5]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
dialogs_doc = nlp(" ".join(dialogs_df.dialogs))

<IPython.core.display.Javascript object>

In [6]:
print("The dialogs_doc object is a {} object.".format(type(dialogs_doc)))
print("It is {} tokens long".format(len(dialogs_doc)))
print("The first three tokens are '{}'".format(dialogs_doc[:3]))
print("The type of each token is {}".format(type(dialogs_doc[0])))

The dialogs_doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 4274276 tokens long
The first three tokens are 'Can we make'
The type of each token is <class 'spacy.tokens.token.Token'>


<IPython.core.display.Javascript object>

In [7]:
# Removing stop words

dialogs_doc_without_stopwords = [token for token in dialogs_doc if not token.is_stop]

<IPython.core.display.Javascript object>

In [8]:
# Applying lemmatization

dialogs_doc_lemma = [token.lemma_ for token in dialogs_doc_without_stopwords]

<IPython.core.display.Javascript object>

In [11]:
postgres_user = "dsbc_student"
postgres_pw = "7*.8G9QH21"
postgres_host = "142.93.121.174"
postgres_port = "5432"
postgres_db = "twitter_sentiment"

engine = create_engine(
    "postgresql://{}:{}@{}:{}/{}".format(
        postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db
    )
)

tweet = pd.read_sql_query("select * from twitter", con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


tweet.head()

,index,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,0,570306133677760513,neutral,1.0000,None,NaN,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2015-02-24 11:35:52 -0800,None,Eastern Time (US & Canada)
1,1,570301130888122368,positive,0.3486,None,0.0000,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2015-02-24 11:15:59 -0800,None,Pacific Time (US & Canada)
2,2,570301083672813571,neutral,0.6837,None,NaN,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,@VirginAmerica it's really aggressive to blast...,None,2015-02-24 11:15:36 -0800,None,Pacific Time (US & Canada)
4,4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2015-02-24 11:14:45 -0800,None,Pacific Time (US & Canada)


<IPython.core.display.Javascript object>

In [13]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
# below is necessary to avoid memory error of SpaCy
nlp.max_length = 20000000

# all the processing work is done below, so it may take a while
tweet_doc = nlp(" ".join(tweet.text))

<IPython.core.display.Javascript object>

In [14]:
print("The tweet_doc object is a {} object.".format(type(tweet_doc)))
print("It is {} tokens long".format(len(tweet_doc)))
print("The first three tokens are '{}'".format(tweet_doc[:3]))
print("The type of each token is {}".format(type(tweet_doc[0])))

The tweet_doc object is a <class 'spacy.tokens.doc.Doc'> object.
It is 307330 tokens long
The first three tokens are '@VirginAmerica What @dhepburn'
The type of each token is <class 'spacy.tokens.token.Token'>


<IPython.core.display.Javascript object>

In [15]:
# Removing stop words

tweet_doc_without_stopwords = [token for token in tweet_doc if not token.is_stop]

<IPython.core.display.Javascript object>

In [16]:
# Applying lemmatization

tweet_doc_lemma = [token.lemma_ for token in tweet_doc_without_stopwords]

<IPython.core.display.Javascript object>